# <h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Importation des bibliothèques et métriques</h1>

In [1]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.gridspec as gridspec
import seaborn as sns
from mpl_toolkits import mplot3d
from scipy import stats
%matplotlib inline

# preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
import pandas_profiling as pp

# models
from sklearn.linear_model import LinearRegression,LogisticRegression, SGDRegressor, RidgeCV
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor
import sklearn.model_selection
from sklearn.model_selection import cross_val_predict as cvp
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import pearsonr

import xgboost as xgb
import lightgbm as lgb

# model tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval

import warnings
warnings.filterwarnings("ignore")

# <h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Chargement du jeux de données</h1>

In [2]:
house = pd.read_csv("dataset/kc_house_data.csv")

# <h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Exploration des données</h1>

# <h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Chargement du jeux de données</h3>

* **id**: identifiant des maisons
* **date**: Date de vente
* **price**: Prix de Vente
* **bedroom**: Nombre de Chambre
* **bathroom**: Nombre de salle de bain
* **sqft_living**: Surface habitable (en pieds carrés)
* **sqf t_lot**: Taille du lot (en pieds carrés)
* **floors**: Nombre d’étages 
* **waterfront**: Maison en front de mer (1 : oui, 0 : non)
* **view**: Variable catégorique de la vue de la maison
* **condition**: Variable catégorique de la condition de la maison
* **grade**: Grade global de la maison basé sur le système de notation du comté de King
* **sqft_above**: Taille de la maison à l’exception du sous-sol (en pieds carrés)
* **sqft_basement**: Taille du sous-sol (en pieds carrés)
* **yr_built**: Année de construction de la maison
* **yr_renovated**: Année de rénovation de la maison (le cas échéant)
* **zipcode**: Code postal de la maison
* **lat**: Latitude de la maison
* **long**: Longitude de la maison
* **sqf t_living15**: Surface habitable en 2015 (en pieds carrés)
* **sqf t_lot15**: Taille du lot en 2015 (en pieds carrés)

# <h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Chargement du jeux de données</h3>

In [3]:
house.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# <h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Vérification de la taille du jeux de données</h3>

In [4]:
house.shape

(21613, 21)

# <h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Vérification du dtype des colonnes ainsi que le nombre de valeurs non nulles dans ces celles-ci</h3>

In [5]:
house.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  


***Nous pouvons remarquer que le jeux de données contient 21613 ligne dont 21 colonnes. Les types présents sont le float64 et le int64***

# <h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Résumé statistique</h3>

In [6]:
house.describe().T

,count,mean,std,min,25%,50%,75%,max
id,21613.0,4.580302e+09,2.876566e+09,1.000102e+06,2.123049e+09,3.904930e+09,7.308900e+09,9.900000e+09
price,21613.0,5.400881e+05,3.671272e+05,7.500000e+04,3.219500e+05,4.500000e+05,6.450000e+05,7.700000e+06
bedrooms,21613.0,3.370842e+00,9.300618e-01,0.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,3.300000e+01
bathrooms,21613.0,2.114757e+00,7.701632e-01,0.000000e+00,1.750000e+00,2.250000e+00,2.500000e+00,8.000000e+00
sqft_living,21613.0,2.079900e+03,9.184409e+02,2.900000e+02,1.427000e+03,1.910000e+03,2.550000e+03,1.354000e+04
sqft_lot,21613.0,1.510697e+04,4.142051e+04,5.200000e+02,5.040000e+03,7.618000e+03,1.068800e+04,1.651359e+06
floors,21613.0,1.494309e+00,5.399889e-01,1.000000e+00,1.000000e+00,1.500000e+00,2.000000e+00,3.500000e+00
waterfront,21613.0,7.541757e-03,8.651720e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
view,21613.0,2.343034e-01,7.663176e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
condition,21613.0,3.409430e+00,6.507430e-01,1.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00


# <h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Visualisation des corrélations entre variables numériques</h3>

In [7]:
plt.figure(figsize=(10,8))
sns.heatmap(house.corr(), cmap="RdBu")
plt.title("Correlations Entre Variables", size=15)
plt.show()

# <h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Sélection des fonctionnalités</h1>


# <h3 style="font-family: 'Times New Roman'"><span class="emphasized">Nous sélectionnons des caractéristiques numériques qui ont un taux de corrélation supérieur à 0,50 ou inférieur à -0,50 basé sur la méthode de corrélation de Pearson, qui est la valeur par défaut du paramètre "méthode" dans la fonction corr(). En ce qui concerne la sélection des caractéristiques catégorielles, j'ai sélectionné les valeurs catégorielles qui, selon moi, ont un effet significatif sur la variable cible.</span>.</h3>
<p style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Le coefficient de Pearson est un indice reflétant une relation linéaire entre deux variables continues. Le coefficient de corrélation varie entre -1 et +1, 0 reflétant une relation nulle entre les deux variables, une valeur négative (corrélation négative) signifiant que lorsqu'une des variable augmente, l'autre diminue ; tandis qu'une valeur positive (corrélation positive) indique que les deux variables varient ensemble dans le même sens. La formule se présente comme suit:</p>

<center><img width="300px" src="images/pearson_correlation.png"></center>

In [8]:
price_col = list(house.corr()["price"][(house.corr()["price"]>0.50) | (house.corr()["price"]<-0.50)].index)
categories_cols = ["date", "bedrooms","sqft_lot","waterfront","view","condition","sqft_basement","yr_built","yr_renovated","zipcode","lat","long","sqft_lot15"]
house_cols = price_col + categories_cols
house = house[house_cols]

In [9]:
price_col


['price', 'bathrooms', 'sqft_living', 'grade', 'sqft_above', 'sqft_living15']

In [10]:
categories_cols

['date',
 'bedrooms',
 'sqft_lot',
 'waterfront',
 'view',
 'condition',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_lot15']

# <h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Vérification des valeurs manquantes</h3>

In [11]:
print("Valeurs manquantes par Colonne")
print("-"*30)
print(house.isna().sum())
print("-"*30)
print("TOTAL VALEURS MANQUANTES:",house.isna().sum().sum())

Valeurs manquantes par Colonne
------------------------------
price            0
bathrooms        0
sqft_living      0
grade            0
sqft_above       0
sqft_living15    0
date             0
bedrooms         0
sqft_lot         0
waterfront       0
view             0
condition        0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_lot15       0
dtype: int64
------------------------------
TOTAL VALEURS MANQUANTES: 0


# <p><u>Remarque</u>: Aucune valeurs manquante. Tout est au complet. Une chance ?</hp>

# <h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Visualisation des données</h1>

***Visualisation de la corrélation entre les variables numériques à l'aide de la visualisation de paires (pairplot)***

In [12]:
sns.pairplot(house[price_col])

# <h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Fractionnement des données en morceaux X et y</h1>

In [13]:
Xhouse_categories = house.drop("price", axis=1)
yhouse_price = house["price"]

<h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Encodage à chaud de house_categories et de house_price</h1>

<h3 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Encodage des caractéristiques catégorielles dans le jeu de données X à l'aide de la méthode One-Hot Encoding</h3>

In [14]:
Xhouse_categories = pd.get_dummies(Xhouse_categories, columns=categories_cols)

<h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Standardisation des données</h1>

***Normalisation des colonnes numériques dans le jeu de données Xhouse_categories. StandardScaler() ajuste la moyenne des caractéristiques à 0 et l'écart type des caractéristiques à 1. La formule utilisée par StandardScaler() est la suivante :***

<center><img width="300px" src="images/standard_deviation.gif"></center>

In [15]:
price_col.remove("price")

scaler = StandardScaler()
Xhouse_categories[price_col] = scaler.fit_transform(X[price_col])

NameError: name 'X' is not defined

In [ ]:
Xhouse_categories.head()

# <h1 style="font-family: 'Times New Roman'; letter-spacing: 0.08em;">Chargement du jeux de données</h1>


In [ ]:
house = pd.read_csv("dataset/kc_house_data.csv")

In [ ]:
house.head()

In [ ]:
house.shape

In [ ]:
house.info

In [ ]:
house.describe().T

In [ ]:
house.describe()['grade']

In [ ]:
house.info()['grade']